In [75]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import json

# IMPORT MAPPING + CREATE DF

In [76]:
#import the classification table
df_mapping = pd.read_excel('Classification_excel.xlsx', index_col=None)
df_mapping["Key words"] = df_mapping["Key words"].str.lower()

#Create df for each of the 4 categories
df_type = df_mapping.loc[df_mapping['Category'] == 'CLA type']
df_status = df_mapping.loc[df_mapping['Category'] == 'CLA status']
df_sector = df_mapping.loc[df_mapping['Category'] == 'Sector']
df_theme = df_mapping.loc[df_mapping['Category'] == 'Theme']

class_type_label_ind = df_type.loc[df_type['Key words'].isnull(), 'Class'].to_string(index=False)
class_type_label_ind

'Concluded on industry level'

# PIPEPLINE PDF --> CREATE LIST TEXT + CLASS

In [77]:
# GO FOR ALL FILE IN 200
# OUTCOME : [pdf unique id, cla type, status, sector, theme : ["salaires"],["durée de travail"]]

In [78]:
all_pdf_titles = [f for f in listdir("200") if isfile(join("200", f))]


In [79]:
dict_pdf = {}
for pdf in all_pdf_titles :
    pdf_class = {}
    pdf_txt = open(f"200/{pdf}", 'r')
    pdf_words = pdf_txt.read().lower()

    # Add the type to the list pdf_class
    class_type_label_cnt = df_type.loc[df_type['Key words'] == "www.cnt-nar.be", 'Class'].to_string(index=False)
    class_type_label_ind = df_type.loc[df_type['Key words'].isnull(), 'Class'].to_string(index=False)

    elements = df_type["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            if class_type_label_cnt not in pdf_class :
                pdf_class["CLA type"] = class_type_label_cnt
                break
        else : 
            if class_type_label_ind not in pdf_class :
                pdf_class["CLA type"] = class_type_label_ind

    # Add the status to the list pdf_class
    class_status_label_update = df_status.loc[df_status['Key words'] == "erratum", 'Class'].to_string(index=False)
    class_status_label_new = df_status.loc[df_status['Key words'].isnull(), 'Class'].to_string(index=False)

    elements = df_status["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            if class_status_label_update not in pdf_class :
                pdf_class["CLA status"] = class_status_label_update
                break
        else : 
            if class_status_label_new not in pdf_class :
                pdf_class["CLA status"] = class_status_label_new

    # Add the sector to the list pdf_class
    class_sector_label = df_sector['Class'].to_list()
    result_2 = []
    elements = df_sector["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            result_2.append(True)
        else : 
            result_2.append(False)

    if True in result_2 :
        pdf_class["Sector"] = df_sector.loc[df_sector["Key words"]== elem, "Class"].to_string(index=False)
    else :
        if "Sector not specified" not in pdf_class :
            pdf_class["Sector"] = "Sector not specified"

    #Add the theme to the list pdf_class

    class_theme_label = df_theme['Class'].to_list()
    result_3 = []
    elements = df_theme["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            result_3.append(True)
        else : 
            result_3.append(False)

    if True in result_3 :
        for elem in elements :
            if elem in pdf_words :
                if df_theme.loc[df_theme["Key words"]== elem, "Class"].to_string(index=False) not in pdf_class :
                    pdf_class["Theme"] = (df_theme.loc[df_theme["Key words"]== elem, "Class"].to_string(index=False))
    else :
        if "Unknown_theme" not in pdf_class :
            pdf_class["Theme"] =  "Unknown_theme"
    dict_pdf[pdf] = pdf_class
dict_pdf

#Changer Quel format vont les pdf arriver --> change "txt"

{'200-2021-013464_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'New',
  'Sector': 'Sector not specified',
  'Theme': 'Primes et indemnités pour le travailleur actif'},
 '200-2021-013468_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'New',
  'Sector': 'Sector not specified',
  'Theme': 'Primes et indemnités pour le travailleur actif'},
 '200-2021-013476_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'New',
  'Sector': 'LA SELLERIE, DE LA FABRICATION DE COURROIES ET ...',
  'Theme': 'Primes et indemnités pour le travailleur actif'},
 '200-2021-013474_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'New',
  'Sector': 'LA SELLERIE, DE LA FABRICATION DE COURROIES ET ...',
  'Theme': 'Primes et indemnités pour le travailleur actif'},
 '200-2021-013466_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'Update',
  'Sector': 'Sector not specified',
  'Theme': 'Fin au contrat de travail e

# Merge with METADATA